## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [ ]:
# from arcgis.gis import GIS
# import contextlib, io
# with contextlib.redirect_stderr(io.StringIO()):
#     gis = GIS("home")

#### Now you are ready to start!

In [ ]:
# # -*- coding: utf-8 -*-
# """
# Title: USGS Daily Water Data to ArcGIS Feature Layer (Notebook Version)
# Description: A script designed for cell-by-cell execution in an ArcGIS Notebook.
#              It fetches daily water data from the USGS Water Services API, processes
#              it into a pandas DataFrame, and updates a target ArcGIS Feature Layer
#              using a robust Truncate and Add workflow with edit_features.
# v10 – Final Production Version.
# """
#
# Optional: ArcGIS Online Sync
# To publish updates to ArcGIS Online, set:
#   USE_ARCGIS=1
#   USGS_WATER_LAYER_ID (or FEATURE_LAYER_ITEM_ID)
# Then re-run the notebook from Cell 1.


In [ ]:
# Cell 1: Install and Import Libraries
# =================================================================================
# This cell ensures the necessary libraries are installed in the notebook environment.
import sys
import subprocess
import logging
import json
import time
import datetime
import os

# Set USE_ARCGIS=1 to enable ArcGIS Online sync; otherwise run locally.
USE_ARCGIS = os.environ.get("USE_ARCGIS", "").lower() in ("1", "true", "yes")

print("Installing required libraries...")
base_pkgs = ["pandas", "openpyxl", "requests"]
if USE_ARCGIS:
    base_pkgs.append("arcgis")
subprocess.check_call([sys.executable, "-m", "pip", "install", *base_pkgs])
print("Installation complete.")

import pandas as pd
import requests
if USE_ARCGIS:
    from arcgis.gis import GIS
    from arcgis.features import Feature
    from arcgis.geometry import Point

# Import display for rich DataFrame output in notebooks
try:
    from IPython.display import display
except ImportError:
    display = print # Fallback to standard print if IPython is not available

# Configure logging to display messages in the notebook output
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    stream=sys.stdout
)

print("Cell 1: Libraries installed and imported.")


In [ ]:
# show me the version
from platform import python_version
print("Python version:", python_version())

pkgs = [pd, logging, requests, json]
if USE_ARCGIS:
    import arcgis
    pkgs.insert(1, arcgis)

for p in pkgs:
    print(f"{p.__name__:-<30}v{p.__version__}")


In [ ]:
# pandas full tables
# import warnings
# warnings.simplefilter(action="ignore", category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# 1️⃣  Busca TODAS las Feature Layers de la organización sin importar el autor (*)
if USE_ARCGIS:
    fl_items = gis.content.search(
        query="*",                  # comodín (puede ser "" o "*")
        item_type="Feature Layer",
        max_items=1000              # ajusta si tienes más
    )

    df = pd.DataFrame([
        {"Title": it.title, "ITEM_ID": it.itemid, "Layers": len(it.layers)}
        for it in fl_items
    ]).sort_values("Title")

    display(df)
else:
    print("ArcGIS disabled; skipping organization search.")


In [ ]:
###########################################

In [ ]:
# %pwd

In [ ]:
# Cell 1: Install and Import Libraries
# =================================================================================
# This cell ensures the necessary libraries are installed in the notebook environment.
import sys
import subprocess
import logging
import json
import time
import datetime
import os

# Set USE_ARCGIS=1 to enable ArcGIS Online sync; otherwise run locally.
USE_ARCGIS = os.environ.get("USE_ARCGIS", "").lower() in ("1", "true", "yes")

print("Installing required libraries...")
base_pkgs = ["pandas", "openpyxl", "requests"]
if USE_ARCGIS:
    base_pkgs.append("arcgis")
subprocess.check_call([sys.executable, "-m", "pip", "install", *base_pkgs])
print("Installation complete.")

import pandas as pd
import requests
if USE_ARCGIS:
    from arcgis.gis import GIS
    from arcgis.features import Feature
    from arcgis.geometry import Point

# Import display for rich DataFrame output in notebooks
try:
    from IPython.display import display
except ImportError:
    display = print # Fallback to standard print if IPython is not available

# Configure logging to display messages in the notebook output
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    stream=sys.stdout
)

print("Cell 1: Libraries installed and imported.")


In [ ]:
# Cell 2: Configuration
# =================================================================================
# --- USER-DEFINED VARIABLES ---

import os
from pathlib import Path

# Set USE_ARCGIS=1 to enable ArcGIS Online sync; otherwise run locally.
USE_ARCGIS = os.environ.get("USE_ARCGIS", "").lower() in ("1", "true", "yes")

def resolve_file(filename, env_var=None, search_roots=None):
    if env_var:
        env_val = os.environ.get(env_var)
        if env_val:
            return env_val
    roots = search_roots or [Path.cwd(), Path.cwd().parent, Path.home()]
    arcgis_home = Path("/arcgis/home")
    if arcgis_home.exists():
        roots.append(arcgis_home)
    for root in roots:
        if root.exists():
            match = next(root.rglob(filename), None)
            if match:
                return str(match)
    raise FileNotFoundError("Set the required env var or place the file under the repo or /arcgis/home.")

# The Item ID of the target Hosted Feature Layer in ArcGIS.
FEATURE_LAYER_ITEM_ID = os.environ.get("USGS_WATER_LAYER_ID") or os.environ.get("FEATURE_LAYER_ITEM_ID")

# The index of the layer within the Feature Layer Collection to be updated.
LAYER_INDEX = 0

if USE_ARCGIS and not FEATURE_LAYER_ITEM_ID:
    raise ValueError("Set USGS_WATER_LAYER_ID (or FEATURE_LAYER_ITEM_ID) in the environment.")

# Resolve local configuration files without hardcoding paths.
DATA_SOURCES_FILE = resolve_file("PR Alert Data Sources.xlsx", env_var="PR_ALERT_XLSX")
PARAMETER_CODE_FILE = resolve_file("parameter_code_codetable.xlsx", env_var="PARAMETER_CODE_XLSX")
SCHEMADEF_FILE = resolve_file("USGS_Daily_Watet_schemadefinitions.xlsx", env_var="USGS_SCHEMA_XLSX")
STATISTIC_ID_FILE = resolve_file("statistic_id_codetable.xlsx", env_var="STATISTIC_ID_XLSX")

# Local outputs (for non-ArcGIS runs)
OUTPUT_DIR = Path(os.environ.get("OUTPUT_DIR", "outputs"))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_CSV = OUTPUT_DIR / "usgs_water.csv"
OUTPUT_GEOJSON = OUTPUT_DIR / "usgs_water.geojson"

# The name of the API source to process from the configuration file.
SOURCE_NAME = "USGS - Daily Water Data (New API)"
PAGE_LIMIT = 250

print("Cell 2: Configuration variables set.")

In [ ]:
# Cell 3: Connect to your ArcGIS Organization (optional)
# =================================================================================
if USE_ARCGIS:
    try:
        logging.info("Connecting to ArcGIS environment...")
        # The "home" parameter uses the credentials of the user running the notebook
        gis = GIS("home")
        logging.info("Successfully connected to %s.", gis.properties.portalHostname)
    except Exception as e:
        logging.error(f"FATAL: Failed to connect to ArcGIS. Error: {e}")
        sys.exit(1)

    print("Cell 3: ArcGIS connection established.")
else:
    gis = None
    logging.info("ArcGIS disabled; running locally only.")


In [ ]:
# Cell 4: Fetch Today's USGS Data (Loyal to Reference Script)
# =================================================================================
logging.info("Fetching data from source configuration...")
try:
    row = pd.read_excel(DATA_SOURCES_FILE).query("Source_Name == @SOURCE_NAME").squeeze()
    api_url = row["URL_Endpoint"]
    
    # This logic now faithfully replicates the user's working script
    api_params = {}
    params_str = row.get("Parameters_or_Selectors")
    if pd.notna(params_str):
        try:
            api_params = json.loads(params_str)
            logging.info("Successfully parsed parameters from Excel.")
        except (json.JSONDecodeError, TypeError):
            logging.error(f"Could not parse 'Parameters_or_Selectors' for {SOURCE_NAME}. Treating as empty.")
            api_params = {}

    api_params.setdefault("limit", PAGE_LIMIT)
    logging.info(f"Page limit set to: {api_params['limit']}")

    if pd.notna(row.get("Bounding_Box")):
        api_params["bbox"] = str(row["Bounding_Box"])
        logging.info(f"Using Bounding Box: {api_params['bbox']}")

    if pd.notna(row.get("API_Key")):
        api_params["api_key"] = str(row["API_Key"])
        logging.info("Using API Key.")

    items, nxt, page = [], api_url, 0
    while nxt:
        page += 1
        logging.info("Fetching page %d…", page)
        # The key is to only send the full parameter set on the FIRST request.
        # Subsequent requests use the 'next' URL which already has the parameters baked in.
        r = requests.get(nxt, params=api_params if page == 1 else {}, timeout=120)
        r.raise_for_status()
        j = r.json()
        items.extend(j.get("features", []))
        nxt = next((l["href"] for l in j.get("links", []) if l.get("rel") == "next"), None)
    
    logging.info("Total fetched: %d", len(items))
    if not items:
        sys.exit("Execution stopped: No data fetched from the API.")

except Exception as e:
    logging.error(f"FATAL: An error occurred during data fetching: {e}")
    import traceback
    traceback.print_exc()
    sys.exit(1)

print("Cell 4: Data fetching complete.")

In [ ]:
# Cell 4.2: Load Parameter Codes definitions
# pandas full tables
# import warnings
# warnings.simplefilter(action="ignore", category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

parameter_code_df = pd.read_excel(PARAMETER_CODE_FILE)

parameter_code_df["parameter_code"] = (
    parameter_code_df["parameter_code"]
    .astype("Int64")     # handles NaNs safely
    .fillna(0)
    .astype(int)
    .astype(str)
    .str.zfill(5)        # ensures 5-digit zero-padding
    .astype("object")    # stored as native Python string
)


# sanity-check
print(parameter_code_df.dtypes)
# limit the number of culumns to: parameter_code	Group Name	Parameter Name/Description
parameter_code_df=parameter_code_df[["parameter_code","Group Name","Parameter Name/Description"]]
display(parameter_code_df)


# pandas full tables
# import warnings
# warnings.simplefilter(action="ignore", category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [ ]:
# Cell 4.1: Load Schema Definitions (To use later)
# pandas full tables
# import warnings
# warnings.simplefilter(action="ignore", category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

schemadef=pd.read_excel(SCHEMADEF_FILE)
display(schemadef)

# pandas full tables
# import warnings
# warnings.simplefilter(action="ignore", category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [ ]:
# Cell 4.3: Load statistic_id definitions
# pandas full tables
# import warnings
# warnings.simplefilter(action="ignore", category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

statistic_id_df = pd.read_excel(STATISTIC_ID_FILE)

statistic_id_df["statistic_id"] = (
    statistic_id_df["statistic_id"]
    .astype("Int64")  # handles NaNs, trims decimals
    .fillna(0)
    .astype(int)
    .astype(str)
    .str.zfill(5)     # pads with leading zeros to 5 digits
    .astype("object") # ensures native Python string type
)

# sanity-check
print(statistic_id_df.dtypes)
display(statistic_id_df)

# pandas full tables
# import warnings
# warnings.simplefilter(action="ignore", category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [ ]:
# Cell 5: Build DataFrame
# =================================================================================
logging.info("Building DataFrame from fetched data...")
try:
    rows = []
    for f in items:
        p, g = f["properties"], f["geometry"]
        lon, lat = (g.get("coordinates") or [None, None])[:2]

        rows.append({
            "geom_type": g.get("type", "Point"),
            "longitude": float(lon) if lon is not None else None,
            "latitude": float(lat) if lat is not None else None,
            "time_series_id": p.get("timeseries_id"),
            "monitoring_location_id": p.get("monitoring_location_id"),
            "value": p.get("value"),
            "approval_status": p.get("approval_status"),
            "last_modified": p.get("last_modified"),
            "parameter_code": p.get("parameter_code"),
            "statistic_id": p.get("statistic_id"),
            "unit_of_measure": p.get("unit_of_measure"),
            "qualifier": p.get("qualifier"),
        })

    df = pd.DataFrame(rows)
    logging.info("Rows in DataFrame: %d", len(df))

    # merge parameter_code
    df=df.merge(parameter_code_df, on="parameter_code", how="left")

    # merge statistic_id
    df=df.merge(statistic_id_df, on="statistic_id", how="left")
    
    # --- Display a sample of the DataFrame ---
    sample_size = 5
    if len(df) > (sample_size * 2):
        logging.info(f"Displaying first and last {sample_size} rows as a sample.")
        display(pd.concat([df.head(sample_size), df.tail(sample_size)]))
    else:
        display(df)

    if not USE_ARCGIS:
        if df.empty:
            logging.info("No records to write locally.")
        else:
            df.to_csv(OUTPUT_CSV, index=False)

            def to_jsonable(val):
                if isinstance(val, pd.Timestamp):
                    return val.isoformat()
                try:
                    if pd.isna(val):
                        return None
                except Exception:
                    pass
                if hasattr(val, "item"):
                    try:
                        return val.item()
                    except Exception:
                        pass
                return val

            features = []
            if {"longitude", "latitude"}.issubset(df.columns):
                for _, row in df.iterrows():
                    lon = row.get("longitude")
                    lat = row.get("latitude")
                    if pd.notna(lon) and pd.notna(lat):
                        props = row.drop(labels=["longitude", "latitude"]).to_dict()
                        props = {k: to_jsonable(v) for k, v in props.items()}
                        features.append({
                            "type": "Feature",
                            "geometry": {"type": "Point", "coordinates": [float(lon), float(lat)]},
                            "properties": props
                        })
            geojson = {"type": "FeatureCollection", "features": features}
            with open(OUTPUT_GEOJSON, "w", encoding="utf-8") as f:
                json.dump(geojson, f, ensure_ascii=False, indent=2)
            logging.info("Local outputs written: %s and %s", OUTPUT_CSV, OUTPUT_GEOJSON)

except Exception as e:
    logging.error(f"FATAL: An error occurred while building the DataFrame: {e}")
    sys.exit(1)

print("Cell 5: DataFrame built successfully.")

In [ ]:
# sanity-check
print(df.dtypes)

In [ ]:
df["Group Name"].sort_values().unique()

In [ ]:
df["Parameter Name/Description"].sort_values().unique()

In [ ]:
df["Stat Nm"].sort_values().unique()

In [ ]:
df["Stat Ds"].sort_values().unique()

In [ ]:
df["qualifier"]

In [ ]:
df["qualifier"].apply(type).value_counts()

In [ ]:
df["qualifier"].head(10).tolist()

In [ ]:
qualifier_norm = df["qualifier"].apply(
    lambda x: tuple(x) if isinstance(x, list) else x   # list → tuple
)

In [ ]:
qualifier_norm

In [ ]:
# 1️⃣  Build a Boolean mask: True only for list-type entries
mask_lists = df["qualifier"].apply(lambda x: isinstance(x, list))

# 2️⃣  Slice the dataframe with that mask
df_lists = df[mask_lists]

# 3️⃣  Take a look
print(f"{df_lists.shape[0]} rows with list qualifiers:")
display(df_lists.head())        # Jupyter friendly; or .to_markdown(), .to_string(), etc.


In [ ]:
list_values = df.loc[mask_lists, "qualifier"].tolist()
for i, val in enumerate(list_values, start=1):
    print(f"{i:>2}: {val}")

In [ ]:
# coerce the lists into simple strings so the column is hash-friendly
def squash_list(x, sep=" - "):
    """
    • If x is a list  ➜  join its items with sep      ['A','B'] → "A - B"
    • Otherwise      ➜  leave it as-is (None, str…)
    """
    return sep.join(x) if isinstance(x, list) else x

df["qualifier"] = df["qualifier"].apply(squash_list)

In [ ]:
df["qualifier"].sort_values().unique()

In [ ]:
# Cell 6: Access Target Layer
# =================================================================================
if not USE_ARCGIS:
    print("ArcGIS disabled; skipping Cell 6.")
else:
    try:
        flayer = gis.content.get(FEATURE_LAYER_ITEM_ID).layers[LAYER_INDEX]
        logging.info("Target layer: %s", flayer.properties.name)
    except Exception as e:
        logging.error(f"FATAL: Could not access target feature layer: {e}")
        sys.exit(1)

    print("Cell 6: Target layer accessed.")


In [ ]:
# Cell 7: Truncate Layer
# =================================================================================
if not USE_ARCGIS:
    print("ArcGIS disabled; skipping Cell 7.")
else:
    try:
        count = flayer.query(return_count_only=True)
        logging.info("Existing feature count: %d", count)
        if count:
            flayer.delete_features(where="1=1")
            logging.info("Layer cleared.")
        else:
            logging.info("Layer already empty.")
    except Exception as e:
        logging.error(f"FATAL: An error occurred during truncation: {e}")
        sys.exit(1)

    print("Cell 7: Truncate operation complete.")


In [ ]:
# Cell 8: Prepare and Push Adds
# =================================================================================
if not USE_ARCGIS:
    print("ArcGIS disabled; skipping Cell 8.")
else:
    try:
        adds = []
        has_geom = bool(getattr(flayer.properties, "geometryType", ""))
    
        # Create a copy to avoid SettingWithCopyWarning
        df_clean = df.copy()

        # --- NEW: Rename DataFrame columns to exactly match Feature Layer field names ---
        column_rename_map = {
            "Group Name": "Group_Name",
            "Parameter Name/Description": "Parameter_Name_Description",
            "Stat Nm": "Stat_Nm",
            "Stat Ds": "Stat_Ds"
        }
        df_clean.rename(columns=column_rename_map, inplace=True)
        logging.info("Renamed DataFrame columns to match feature layer schema.")
        # -----------------------------------------------------------------------------
    
        for _, row in df_clean.iterrows():
            # --- Create a dictionary of all attributes from the row ---
            attrs = row.to_dict()
        
            # --- Date Handling (DO NOT CHANGE) ---
            ts_object = pd.to_datetime(attrs.get('last_modified'), utc=True, errors='coerce')

            if pd.notna(ts_object):
                attrs['last_modified'] = ts_object.strftime('%Y-%m-%d %H:%M:%S')
                attrs['time'] = ts_object.strftime('%Y-%m-%d')
            else:
                attrs['last_modified'] = None
                attrs['time'] = None
        
            # --- Explicit Type Casting for Problem Fields ---
            attrs['longitude'] = float(attrs['longitude']) if pd.notna(attrs.get('longitude')) else None
            attrs['latitude'] = float(attrs['latitude']) if pd.notna(attrs.get('latitude')) else None
            attrs['value'] = float(attrs['value']) if pd.notna(attrs.get('value')) else None
            attrs['parameter_code'] = int(attrs['parameter_code']) if pd.notna(attrs.get('parameter_code')) else None
            attrs['statistic_id'] = int(attrs['statistic_id']) if pd.notna(attrs.get('statistic_id')) else None
        
            # Cast all text fields to string to be safe, now using the RENAMED column names
            attrs['time_series_id'] = str(attrs['time_series_id']) if pd.notna(attrs.get('time_series_id')) else None
            attrs['qualifier'] = str(attrs['qualifier']) if pd.notna(attrs.get('qualifier')) else None
            attrs['Group_Name'] = str(attrs.get('Group_Name')) if pd.notna(attrs.get('Group_Name')) else None
            attrs['Parameter_Name_Description'] = str(attrs.get('Parameter_Name_Description')) if pd.notna(attrs.get('Parameter_Name_Description')) else None
            attrs['Stat_Nm'] = str(attrs.get('Stat_Nm')) if pd.notna(attrs.get('Stat_Nm')) else None
            attrs['Stat_Ds'] = str(attrs.get('Stat_Ds')) if pd.notna(attrs.get('Stat_Ds')) else None
        
            # Final cleanup of any remaining None/NaN values
            clean_attrs = {k: v for k, v in attrs.items() if v is not None}

            # --- Geometry Creation ---
            if has_geom and pd.notna(row["longitude"]) and pd.notna(row["latitude"]):
                geom = Point({"x": row["longitude"], "y": row["latitude"], "spatialReference": {"wkid": 4326}})
                adds.append(Feature(geometry=geom, attributes=clean_attrs))
            else:
                adds.append(Feature(attributes=clean_attrs))

        # --- Final Pre-flight Check (Commented out for production runs) ---
        # if adds:
        #     print("\n" + "="*25 + " DEBUG: FIRST FEATURE OBJECT TO BE SENT " + "="*25)
        #     # Correctly serialize the Feature object's dictionary for printing
        #     print(json.dumps(adds[0].as_dict, indent=2))
        #     print("="*75 + "\n")

        logging.info("Prepared %d features for add.", len(adds))

        # --- Pushing the Edits ---
        if adds:
            result = flayer.edit_features(adds=adds, rollback_on_failure=True)
            # The detailed response is now commented out for cleaner production logs.
            # logging.info("edit_features response:\n%s", json.dumps(result, indent=2))

            ok = all(r.get("success") for r in result.get("addResults", []))
            if not ok:
                raise RuntimeError("Some features failed to add – see response above.")

            logging.info("✔ All features added (%d)", len(adds))
        else:
            logging.info("No features to add.")

    except Exception as e:
        logging.error(f"FATAL: An error occurred while preparing or pushing edits: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)

    print("--- WORKFLOW COMPLETE ---")
